In [ ]:
import pandas as pd
import numpy as np
from src import extraer_coordenadas_manual, insertar_datos_automatico, conexion_postgres
import psycopg2
pd.set_option('display.max_columns', None)
import os
from dotenv import load_dotenv
import requests

In [ ]:
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")

In [ ]:
dict_id_tipo_evento = {}
id_tipo_evento = 1
for x in df_eventos["type"].unique().tolist():
    dict_id_tipo_evento[x] = id_tipo_evento
    id_tipo_evento = id_tipo_evento + 1


In [ ]:
df_eventos["type_id"] = df_eventos["type"].map(dict_id_tipo_evento)

In [ ]:
df_eventos.drop(columns=(['bad_behaviour_card',
 'off_camera',
 'related_events',
 'substitution_outcome',
 'substitution_outcome_id',
 'substitution_replacement',
 'substitution_replacement_id',
 'shot_freeze_frame',
 '50_50',
 'index',
 'duration',
 'tactics',
 "pass_angle",
 "pass_length",
 "minute",
 "second",
 'injury_stoppage_in_chain',
 'player_off_permanent',
 'half_end_early_video_end']), inplace=True)

In [ ]:
lista_col_location = []
for col in df_eventos.columns:
    if "location" in col:
        lista_col_location.append(col)
for loc_col_name in lista_col_location:
    # Nombres para las nuevas columnas de x e y
    nueva_col_x = f"{loc_col_name}_x"
    nueva_col_y = f"{loc_col_name}_y"
    nueva_col_z = f"{loc_col_name}_z"
    # Aplicar la función y asignar a las nuevas columnas
    df_eventos[[nueva_col_x, nueva_col_y, nueva_col_z]] = df_eventos[loc_col_name].apply( lambda val: pd.Series(extraer_coordenadas_manual(val)))
lista_drop = []
for drop in df_eventos.columns:
    if drop.startswith("shot"):
        continue
    if "location" in drop and drop.endswith("z"):
        lista_drop.append(drop)
df_eventos.drop(columns=lista_drop, inplace=True)
df_eventos.drop(columns=lista_col_location, inplace=True)

In [ ]:
df_eventos = pd.read_csv("data/df_eventos_liga2015_mod.csv")

C:\Users\pepel\AppData\Local\Temp\ipykernel_18388\1583794950.py:1: DtypeWarning: Columns (0,2,9,10,16,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,58,60,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eventos = pd.read_csv("data/df_eventos_liga2015_mod.csv")


In [4]:
columnas_bool = []

for col in df_eventos.columns:
    if df_eventos[col].apply(lambda x: isinstance(x, bool) and x is True).any():
        df_eventos[col] = df_eventos[col].fillna(False)
        columnas_bool.append(col)

C:\Users\pepel\AppData\Local\Temp\ipykernel_18388\2590581491.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_eventos[col] = df_eventos[col].fillna(False)


In [ ]:
type(df_eventos["carry_end_location_x"][0])

In [ ]:
df_eventos.dtypes

In [6]:
for columna in df_eventos.columns:
    if df_eventos[columna].dtype in ['int64', 'float64']:
        min_val = df_eventos[columna].min()
        max_val = df_eventos[columna].max()
        print(f"Columna {columna}: Min = {min_val}, Max = {max_val}, Type {type(df_eventos[columna])}")

Columna match_id: Min = 265839, Max = 3825908, Type <class 'pandas.core.series.Series'>
Columna pass_recipient_id: Min = 3023.0, Max = 220761.0, Type <class 'pandas.core.series.Series'>
Columna period: Min = 1, Max = 2, Type <class 'pandas.core.series.Series'>
Columna player_id: Min = 3023.0, Max = 220761.0, Type <class 'pandas.core.series.Series'>
Columna possession: Min = 1, Max = 250, Type <class 'pandas.core.series.Series'>
Columna possession_team_id: Min = 207, Max = 1049, Type <class 'pandas.core.series.Series'>
Columna shot_statsbomb_xg: Min = 0.00018, Max = 0.9800704, Type <class 'pandas.core.series.Series'>
Columna team_id: Min = 207, Max = 1049, Type <class 'pandas.core.series.Series'>
Columna type_id: Min = 1, Max = 33, Type <class 'pandas.core.series.Series'>
Columna carry_end_location_x: Min = 0.1, Max = 120.2, Type <class 'pandas.core.series.Series'>
Columna carry_end_location_y: Min = 0.1, Max = 80.0, Type <class 'pandas.core.series.Series'>
Columna goalkeeper_end_locati

In [7]:
lista_cols_int = ["match_id", "pass_recipient_id", "period", "player_id", "possession", "possession_team_id", "team_id", "type_id"]
for x in lista_cols_int:
    df_eventos[x] = df_eventos[x].astype('Int64')
    df_eventos[x].replace({pd.NA: None}, inplace=True)

C:\Users\pepel\AppData\Local\Temp\ipykernel_18388\135172166.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_eventos[x].replace({pd.NA: None}, inplace=True)
C:\Users\pepel\AppData\Local\Temp\ipykernel_18388\135172166.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [8]:
df_eventos.replace({pd.NA: None}, inplace=True)

In [13]:
df_prueba = df_eventos.copy()

In [15]:
lista_cols_nan = []
for col in df_prueba.columns:
    # Verificamos si hay al menos un np.nan (de tipo float y que sea np.nan)
    # o si hay pd.NA (que puede aparecer en DataFrames con tipos de datos que soportan NA, como Int64, StringDtype)
    contiene_npnan = df_prueba[col].apply(lambda x: isinstance(x, float) and pd.isna(x)).any()
    contiene_pdna = df_prueba[col].isna().any() # Verifica pd.NA para cualquier tipo

    if contiene_npnan or contiene_pdna:
        # Convertimos la columna a tipo 'object' para poder mezclar tipos (float, str, None)
        # Esto es necesario si la columna originalmente no es 'object' y queremos insertar None
        df_prueba[col] = df_prueba[col].astype(object)

        # Reemplazamos np.nan y pd.NA por None
        # Importante: inplace=True hace que el método devuelva None, lo cual asignaría None a toda la columna.
        # Por eso, se debe eliminar inplace=True y asignar el resultado de la operación.
        df_prueba[col] = df_prueba[col].replace({np.nan: None, pd.NA: None})

        lista_cols_nan.append(col)

In [16]:
df_prueba["pass_end_location_x"].unique()

array([None, 62.1, 59.1, ..., 120.4, 120.3, 120.1],
      shape=(1206,), dtype=object)

In [ ]:
for columna in df_eventos.columns:
    if df_eventos[columna].dtype in ['int64', 'float64']:
        min_val = df_eventos[columna].min()
        max_val = df_eventos[columna].max()
        print(f"Columna {columna}: Min = {min_val}, Max = {max_val}, Type {type(df_eventos[columna])}")

In [ ]:
df_eventos["player_id"]

In [17]:
df_eventos.rename(columns=({"player_id":"playerid",
                            "player": "nombre_completo_jugador",
                            "team_id": "clubid",
                            "team": "club"}), inplace=True)

In [18]:
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_eventos,"eventos", cur, conn)

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)


In [19]:
df_eventos.to_csv("data/Tabla_Eventos.csv", index=False)